In [1]:
import pandas as pd
import datetime as dt
from meteostat import Point, Hourly
import pymysql

In [2]:
collision = pd.read_csv("NYC_Collisions.csv")

In [3]:
collision.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238421 entries, 0 to 238420
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Collision ID         238421 non-null  int64  
 1   Date                 238421 non-null  object 
 2   Time                 238421 non-null  object 
 3   Borough              231224 non-null  object 
 4   Street Name          238058 non-null  object 
 5   Cross Street         111291 non-null  object 
 6   Latitude             216098 non-null  float64
 7   Longitude            216098 non-null  float64
 8   Contributing Factor  237134 non-null  object 
 9   Vehicle Type         238421 non-null  object 
 10  Persons Injured      238420 non-null  float64
 11  Persons Killed       238421 non-null  int64  
 12  Pedestrians Injured  238421 non-null  int64  
 13  Pedestrians Killed   238421 non-null  int64  
 14  Cyclists Injured     238421 non-null  int64  
 15  Cyclists Killed  

In [4]:
collision.isna().any()

Collision ID           False
Date                   False
Time                   False
Borough                 True
Street Name             True
Cross Street            True
Latitude                True
Longitude               True
Contributing Factor     True
Vehicle Type           False
Persons Injured         True
Persons Killed         False
Pedestrians Injured    False
Pedestrians Killed     False
Cyclists Injured       False
Cyclists Killed        False
Motorists Injured      False
Motorists Killed       False
dtype: bool

In [5]:
collision["Time"]=collision["Time"].str[:2]
collision["Date"]=collision["Date"]+ " "+ collision["Time"]

In [6]:
collision["Date"] = pd.to_datetime(collision["Date"], format="%Y-%m-%d %H")

In [7]:
collision["Persons Injured"] = collision["Persons Injured"].fillna(0).astype(int)
collision["Borough"] = collision["Borough"].fillna(value="Unknown")
collision["Street Name"] = collision["Street Name"].fillna(value="Unknown")
collision["Contributing Factor"] = collision["Contributing Factor"].fillna(value="Unknown")

In [8]:
# df for weather data import
weather=collision[["Collision ID","Date", "Latitude", "Longitude"]]

In [9]:
#if latitude or Longitude is NaN, I can't import weather data, so I drop it
weather=weather.dropna(how="any")
weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 216098 entries, 0 to 238420
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Collision ID  216098 non-null  int64         
 1   Date          216098 non-null  datetime64[ns]
 2   Latitude      216098 non-null  float64       
 3   Longitude     216098 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 8.2 MB


In [10]:
collision=collision.drop(columns=["Time","Cross Street", "Latitude", "Longitude"])

In [11]:
collision.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238421 entries, 0 to 238420
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Collision ID         238421 non-null  int64         
 1   Date                 238421 non-null  datetime64[ns]
 2   Borough              238421 non-null  object        
 3   Street Name          238421 non-null  object        
 4   Contributing Factor  238421 non-null  object        
 5   Vehicle Type         238421 non-null  object        
 6   Persons Injured      238421 non-null  int32         
 7   Persons Killed       238421 non-null  int64         
 8   Pedestrians Injured  238421 non-null  int64         
 9   Pedestrians Killed   238421 non-null  int64         
 10  Cyclists Injured     238421 non-null  int64         
 11  Cyclists Killed      238421 non-null  int64         
 12  Motorists Injured    238421 non-null  int64         
 13  Motorists Kill

In [12]:
#to avoid problems with importing strings into the database, there can be no '
collision['Street Name'] = collision['Street Name'].str.replace("'", "")

In [13]:
#number of character in columns with string (info for database)
collision["Borough"].str.len().max() #13
collision["Borough"].str.len().min() #5

collision["Street Name"].str.len().max() #32
collision["Street Name"].str.len().min() #2

collision["Contributing Factor"].str.len().max() #53
collision["Contributing Factor"].str.len().min() #5

collision["Vehicle Type"].str.len().max() #22
collision["Vehicle Type"].str.len().min() #3

3

In [14]:
#function to import weather data
def get_weather(date, lat, lon):
    #define location
    location = Point(lat, lon)

    #fetch data
    start = date
    end = date
    data = Hourly(location, start, end)
    data = data.fetch()

    if not data.empty:
        # Extract the weather data
        temp = data["temp"].iloc[0] if "temp" in data.columns else None
        humidity = data["rhum"].iloc[0] if "rhum" in data.columns else None
        precipitation = data['prcp'].iloc[0] if 'prcp' in data.columns else None
        wind_speed = data['wspd'].iloc[0] if 'wspd' in data.columns else None
        air_pressure = data['pres'].iloc[0] if 'pres' in data.columns else None
        return temp, humidity, precipitation, wind_speed, air_pressure,
    else:
        return None, None, None, None, None

In [15]:
#import weather data
weather[["Temperature", "Humidity", "Precipitation", 'Wind speed', 'Air pressure']] = weather.apply(
    lambda row: pd.Series(get_weather(row['Date'], row['Latitude'], row['Longitude'])),
    axis=1)

In [16]:
weather

,Collision ID,Date,Latitude,Longitude,Temperature,Humidity,Precipitation,Wind speed,Air pressure
0,4491746,2021-01-01 20:00:00,40.833980,-73.826350,3.0,70.0,0.0,0.0,1029.1
1,4441905,2021-01-01 05:00:00,40.687300,-73.973656,0.6,66.0,0.0,11.2,1029.8
3,4380949,2021-01-01 19:00:00,40.882700,-73.892730,4.0,61.0,0.0,7.6,1029.4
4,4380940,2021-01-01 07:00:00,40.637910,-73.978640,0.0,72.0,0.0,7.6,1031.2
5,4380963,2021-01-01 18:00:00,40.848630,-73.927620,4.0,61.0,0.0,7.6,1030.5
...,...,...,...,...,...,...,...,...,...
238416,4619581,2023-04-09 04:00:00,40.715443,-73.951850,5.3,66.0,0.0,10.4,1031.6
238417,4619685,2023-04-09 08:00:00,40.746864,-73.877170,5.0,55.0,0.0,7.9,1032.0
238418,4619519,2023-04-09 21:00:00,40.642017,-73.962660,10.6,41.0,0.0,22.3,1032.1
238419,4619921,2023-04-09 11:00:00,40.763790,-73.989655,6.7,47.0,0.0,11.2,1032.6


In [17]:
weather=weather.dropna(how="any")
weather=weather.drop(columns=["Date"])
weather=weather.reset_index(drop=True)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216076 entries, 0 to 216075
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Collision ID   216076 non-null  int64  
 1   Latitude       216076 non-null  float64
 2   Longitude      216076 non-null  float64
 3   Temperature    216076 non-null  float64
 4   Humidity       216076 non-null  float64
 5   Precipitation  216076 non-null  float64
 6   Wind speed     216076 non-null  float64
 7   Air pressure   216076 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 13.2 MB


In [18]:
weather.to_csv('weather.csv')
collision.to_csv('collision.csv')

In [19]:
conn = pymysql.connect(database="collision_database", user="root", password="Password!1")
cursor=conn.cursor()

In [20]:
cursor.execute("SELECT * FROM collision_database.collision;")

0

In [21]:
insert_query="INSERT INTO collision_database.collision(collision_ID, date_time, borough, street_name, contributing_factor, vehicle_type, persons_injured, persons_killed, pedestrians_injured, pedestrians_killed, cyclists_injured, cyclists_killed, motorists_injured, motorists_killed) VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s)"
insert_query

'INSERT INTO collision_database.collision(collision_ID, date_time, borough, street_name, contributing_factor, vehicle_type, persons_injured, persons_killed, pedestrians_injured, pedestrians_killed, cyclists_injured, cyclists_killed, motorists_injured, motorists_killed) VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s)'

In [22]:
data = [tuple(row) for row in collision.itertuples(index=False, name=None)]

In [23]:
def insert_records_in_chunks(data, chunk_size=1000):
    for i in range(0, len(data), chunk_size):
        chunk = data[i:i + chunk_size]
        cursor.executemany(insert_query, chunk)
        conn.commit()

In [24]:
insert_records_in_chunks(data)

In [25]:
cursor.execute("SELECT * FROM collision_database.weather;")

0

In [26]:
insert_query_weather="INSERT INTO collision_database.weather(collision_ID, latitude, longitude, temperature, humidity, precipitation, wind_speed, air_pressure) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
insert_query_weather

'INSERT INTO collision_database.weather(collision_ID, latitude, longitude, temperature, humidity, precipitation, wind_speed, air_pressure) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'

In [27]:
data_weather = [tuple(row) for row in weather.itertuples(index=False, name=None)]

In [28]:
def insert_records_in_chunks_weather(data_weather, chunk_size=1000):
    for i in range(0, len(data_weather), chunk_size):
        chunk_weather = data_weather[i:i + chunk_size]
        cursor.executemany(insert_query_weather, chunk_weather)
        conn.commit()

In [29]:
insert_records_in_chunks_weather(data_weather)

In [30]:
cursor.close()
conn.close()